# Tarea 4.1 **Extraccion de informacion desde una pagina web.**

### Ejercicio 1. Consumir una API

Selecciona una API pública y extrae datos utilizando Python.

#### 1. Instalo la libreria de yahoo finance

In [ ]:
#pip install yfinance

#### 2.Creo el codigo

In [ ]:
import yfinance as yf
import pandas as pd

def obtener_datos_api(ticker, fecha_inicial, fecha_final):
    try:
        # Descargar información del ticker
        accion = yf.Ticker(ticker)
        
        # Obtener datos históricos entre las fechas especificadas
        datos_historicos = accion.history(start=fecha_inicial, end=fecha_final)
        
        # Crear un DataFrame con la información relevante
        tabla = pd.DataFrame({
            'Fecha': datos_historicos.index,  # Fechas
            'Cierre': datos_historicos['Close'].round[2],  # Precios de cierre
        })
        
        # Formatear la columna Fecha al formato 'yyyy-mm-dd'
        tabla['Fecha'] = pd.to_datetime(tabla['Fecha']).dt.strftime('%Y-%m-%d')
        
        # Añadir la información adicional sobre la empresa
        Nombre = accion.info.get('longName')
        
        # Obtener la capitalización de mercado actual
        capitalizacion_mercado = accion.info.get('marketCap')
        
        # Imprimir capitalización de mercado fuera de la tabla
        print(f"Capitalización de Mercado: {capitalizacion_mercado}")
        print(f"Nombre de la empresa: {Nombre}") 
        
        return tabla.reset_index(drop=True)  # Reinicio el índice para evitar que la fecha esté como índice y la info aparezca duplicada
    except Exception as e:
        print(f"Error al obtener datos de la API: {e}")
        return None

##### 3.Hago una prueba para ver si funciona.

In [ ]:
ticker = 'AAPL'  # Ticker de Apple
fecha_inicial = '2024-06-01'  # Fecha inicial
fecha_final = '2024-10-12'    # Fecha final

datos = obtener_datos_api(ticker, fecha_inicial, fecha_final)
datos

## Ejercicio 2. **Obtener datos con Web Scraping**

Selecciona un sitio web y extrae datos utilizando la técnica de Web Scraping en Python.

#### 2. Creo el codigo.

Intente replicar la informacion que se podia sacar con la API pero me fue imposible, no se como mejorar la variable **URL= f"https://finance.yahoo.com/quote/{ticker}"** para que permita extraer la informacion de todos los dias, imagino que la pagina de yahoo tiene restricciones para la extraccion de informacion fuera del uso de la API. Tengo que aprender mas acerca de como simular mejor las extracciones de informacion sin el uso de APIs.

In [ ]:
import requests
from bs4 import BeautifulSoup

def obtener_precio_hoy(ticker):
    try:
        # URL de Yahoo Finance para el ticker especificado
        url = f"https://finance.yahoo.com/quote/{ticker}"
        
        # Encabezados HTTP para imitar una solicitud de navegador
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # Hacer la solicitud HTTP a la página web
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            raise Exception(f"Error HTTP {response.status_code}: no se pudo acceder a Yahoo Finance")
        
        # Analizar el contenido HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Buscar el precio actual en la página (usualmente está en un span con estos atributos)
        precio_actual = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'})
        if precio_actual is None:
            raise Exception("No se encontró el precio actual en la página")

        # Convertir el precio a un número de tipo float
        precio = float(precio_actual.text.replace(',', ''))

        return precio
    except Exception as e:
        print(f"Error al obtener el precio de la acción: {e}")
        return None

#### 3. Hago una prueba a ver si funciona.

In [ ]:
ticker = 'AAPL'  # Ticker de Apple
precio_hoy = obtener_precio_hoy(ticker)
if precio_hoy:
    print(f"El precio de la acción de {ticker} hoy es: {precio_hoy}")